#### Project 1: Differential Expression Analysis (transcriptomics)


The goal for this project is to investigate differences in gene expression between samples of healthy lung tissues and samples of idiopathic pulmonary fibrosis (IPF) lung tissue.
 - ([Data here](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE150910))
 -  ([Paper here](https://pmc.ncbi.nlm.nih.gov/articles/PMC7667907/))

**DISCLAIMER:**
the purpose of this analysis is to be educational and simplistic, not to follow current best-practices for differential expression analysis. If you're doing this for real as part of a research project, you should probably use an R package like ([DESeq2](https://bioconductor.org/packages/release/bioc/html/DESeq2.html)),([edgeR](https://bioconductor.org/packages/release/bioc/html/edgeR.html)) , or ([Limma-Voom](https://ucdavis-bioinformatics-training.github.io/2018-June-RNA-Seq-Workshop/thursday/DE.html)) and follow the standard analysis pipelines.

In [1]:
%%capture
%pip install scipy

In [2]:
import numpy as np
import csv
import random
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp
import seaborn as sns

#### Step 1: Reading and processing data

**Note:** This data is not included on this Github repo (because it's not my dataset) but can be downloaded for free on the ([Gene Expression Omnibus website here](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE150910)). The specific file we need is called GSE150910_gene-level_count_file.csv.

We'll store the gene names in a list called `genes`, and we'll store the expression measurements for each gene in a list of lists called `data`.

In [3]:
genes = []
data = []
with open("./data/GSE150910_gene-level_count_file.csv") as csvfile:
  reader = csv.reader(csvfile, delimiter=',')
  for row in reader:
    genes.append(row[0])
    data.append(row[1:])

The first row was the sample labels, so we'll save that as its own list called `samples`, and then remove it from `data` and `genes`.


In [4]:
samples = data[0]
data = data[1:]
genes = genes[1:]

Next, we'll cast everything to NumPy arrays, since those will be easier to work with than lists. And we also need to cast the elements of `data` to floats, since they were read in as strings by default.

In [5]:
data = np.array(data).astype(float)
genes = np.array(genes)
samples = np.array(samples)

Quick check to make sure the shapes of these arrays make sense...

In [6]:
print(genes.shape)
print(samples.shape)
print(data.shape)


(18838,)
(288,)
(18838, 288)


Quick check to make sure the content of these arrays looks of...

In [7]:
print("genes:")
print(genes[0:5])
print("samples:")
print(samples[0:5])
print("data:")
print(data[0:5])

genes:
['TSPAN6' 'TNMD' 'DPM1' 'SCYL3' 'C1orf112']
samples:
['chp_26' 'chp_31' 'chp_34' 'chp_38' 'chp_1']
data:
[[1361.  993.  351. ...  465.  639.  944.]
 [   5.   13.    0. ...    0.    0.    0.]
 [1929. 2775. 1894. ...  860. 1387. 2150.]
 [ 176.  216.  208. ...   21.  146.  204.]
 [  93.  143.   97. ...   72.   69.  158.]]


Okay, next we want to make an array with the experimental condition labels for each sample. We can do this with some pretty basic string manipulation.

In [8]:
labels = []

for i in range(len(samples)):
  tmp = samples[i].split("_")
  labels.append(tmp[0])
    
labels = np.array(labels)

Quick checks..

In [9]:
print(labels[0:5])
print(np.unique(labels))

['chp' 'chp' 'chp' 'chp' 'chp']
['chp' 'control' 'ipf']


Next, we'll get rid of the chronic hypersensitivity pneumonitis(CHP) samples so that we're left with only the control and idiopathic pulmonary fibrosis(IPF) samples.

In [10]:
data = data[:  ,labels != "chp"]
samples = samples[labels != "chp"]
labels = labels[labels != "chp"]


quick sanity check again...

In [11]:
print(data.shape)
print(samples.shape)
print(labels.shape)

(18838, 206)
(206,)
(206,)


The total number of counts in a sample is called **sequencing depth**. We want to control for the difference in sequencing depth between samples, in case the experiment happened to be more sensitive in some samples and less in others.
To do this, we will apply a counts per million(CPM) normalization. For each sample, we'll divide the counts for each gene by the total number of counts in that sample, then multiply by a million.



In [12]:
print(data.shape[1])

206


In [13]:
for j in range(data.shape[1]):
  columnSum = sum(data[:, j])
  
  data[:,j] = data[:,j] / columnSum * 1000000

In [14]:
print(data)

[[2.62026354e+01 3.98260770e+01 3.24569396e+01 ... 3.31761577e+01
  3.12351859e+01 3.07286955e+01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.82286581e-01
  0.00000000e+00 0.00000000e+00]
 [7.37006988e+01 8.12189622e+01 5.73764856e+01 ... 6.33142058e+01
  6.77984395e+01 6.99859061e+01]
 ...
 [1.23451757e-01 2.91499191e-01 1.92280448e-01 ... 3.03810968e-02
  1.46644065e-01 1.62757921e-01]
 [1.82091341e+00 2.76924232e+00 1.42287531e+00 ... 1.79248471e+00
  1.80861014e+00 2.05074981e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


This is one common way of normalization gene expression data. Some other methods also control for gene length (which we don't have for this dataset) in addition to sequencing depth. 
Next we'll apply a filter to eliminate genes that had very low expression in both conditions.

In [15]:
mean_CPM_Control = data[:,labels=="control"].mean(axis=1)
mean_CPM_ipf = data[:, labels == "ipf"].mean(axis=1)

toKeep = (mean_CPM_ipf >= 5) | (mean_CPM_Control >= 5)

data=data[toKeep, :]
genes = genes[toKeep]

print(data.shape)
print(len(genes))


(11097, 206)
11097


**NOTE:** in a real-life research project, you might want to apply additional quality checks and normalizations at this step -- batch correction, correction based on demographic info about subjects (sex, race, age, health status, etc), identifying outliers and thinking about if you should remove them or not. For the purposes of this class, we'll skip these steps, but in a real project this would be the time to do them.

#### Step 2: Differential expression analysis

Now, we'll check each gene to see if it's expression is different in the control and IPF conditions. The standard R liraries for differential expression analysis use complex statistical models to check this, but for our educational purposes, we'll just use a ([Kolmogorv-Smirnov test](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test)). The KS test is non-paremetric, meaning that it doesn't make any assumptions about the distribution being compared.

Note that using the KS test here is unrealistic and it's not typically used in real analysis pipelines,not because it's incorrect, but because it's underpowered to detect true differences in expression compared to the more clever statistical models. But the goal for this class isn't to follow a realistic analysis pipeline, it's to explore this dataset in a way that every step of the analysis makes sense to us, as an educational exercise.

Ok, let's try to actually do this differential expression check now. We're eventually going to iterate over the dataset and check every gene, but let's just start with one gene (the first, at index 0) for now so we can figure out what we need to do.

To start, we'll split the distribution into two separate arrays: one with the gene expression for control samples, and the other with gene expression for the IPF samples:


In [16]:
#starting with just the first gene. We'll eventually do this for every gene
control = data[0, labels == "control"]
ipf = data[0, labels == "ipf"]

Quick shape check...

In [17]:
print(control.shape)
print(ipf.shape)

(103,)
(103,)


Next, we'll actually perform thr KS-test, using `ks_2samp` function that we imported from `scipy.stats` at the top, and remembering to add our `epsilon` value to every element and apply the log2 transformation. This gives us two values: a test statistic and a p-value: 

In [18]:
from scipy.stats import ks_2samp

In [19]:
from scipy.stats import ks_2samp
ksStatistics,pValue = ks_2samp(control, ipf)

print(f"ks_statistic: {ksStatistics}")
print(f"p_value: {pValue}")

ks_statistic: 0.4368932038834951
p_value: 3.264129457102852e-09


Again, this is just a test of whether the distriubtions are different, and doesn't tell us whether or not there's a difference in means (which is what we're really interested in). So besides getting the p-value for the statistical test, we also want to calculate something called the $\log_2$(fold change) between the mean expression level in each condition:

$$
\log_2 \left( \frac{\text{Expression Mean (IPF)}}{\text{Expression Mean (Control)}} \right)
$$

The inside fraction $\frac{\text{Expression Mean (IPF)}}{\text{Expression Mean (Control)}}$ is the fold change. This tells us how different the mean expression of the IPF samples is compared to the control samples. The reason we're using $\log_2$ is to make this measurement symetrical around 0. So if a gene's mean expression in IPF samples is *twice* the mean of the control samples, then $\log_2 \left( \frac{\text{Expression Mean (IPF)}}{\text{Expression Mean (Control)}} \right) = 1$. If it's *half* the mean of the control samples, then $\log_2 \left( \frac{\text{Expression Mean (IPF)}}{\text{Expression Mean (Control)}} \right) = -1$.

One more thing before we actually calculate this -- if either of the means is 0, that will cause a problem for us. We can't divide by 0, and we also can't do $\log_2(0)$. We'll deal with this by defining a small constant called `epsilon` (often 1, 0.1, or 0.01) and adding it to both means before doing the calculation.

Here's how this calculation is done, again using the first gene (index 0) as an example:

In [20]:
epsilon = 1 #small constant to avoid division by zero

controlMean = np.mean(control)
ipfMean = np.mean(ipf)

# calculate log2(fold change), adding epsilon to avoid division by 0
FC = (ipfMean+epsilon) / (controlMean+epsilon)
log2_FC = np.log2(FC)
print(log2_FC)

0.7894289821093974


ok, that's basically it! Now we just need to go through and do this to every gene, and save `p_value` and `log2_FC` for each one.

In [21]:
#getting p-value and log2FC, now for every gene

p_values = []
log2_FCs = []

epsilon = 1

for i in range(data.shape[0]):
  control = data[i, labels == "control"]
  ipf = data[i, labels == "ipf"]
  
  # perform ks-test
  ksStatistics, pValue = ks_2samp(control,ipf)
  
  # save p_value for this gene
  p_values.append(pValue)
  
  # calculate means for each condition
  controlMean = np.mean(control)
  ipfMean = np.mean(ipf)
  
  # calculate log2(fold change), adding epsilon to avoid division by 0
  FC = (ipfMean+epsilon) / (controlMean+epsilon)
  log2_FC = np.log2(FC)
  
  # save log2_Fc for this gene
  log2_FCs.append(log2_FC)
  
p_values = np.array(p_values)
log2_FCs = np.array(log2_FCs)

In [22]:
print(p_values)

[3.26412946e-09 1.79707436e-02 6.02011471e-01 ... 2.28643986e-05
 2.95016347e-04 1.14321993e-05]


Next we'll do a [Bonferroni correction](https://en.wikipedia.org/wiki/Bonferroni_correction), multiplying each p-value by the number of tests we did. 

In [23]:
p_values_bonf = p_values * len(genes)

In [24]:
print(p_values_bonf)

[3.62220446e-05 1.99421342e+02 6.68052129e+03 ... 2.53726232e-01
 3.27379640e+00 1.26863116e-01]




(**NOTE**: in practice, people typically use another multiple-test correction called the [ Benjamini-Hochberg correction](https://www.youtube.com/watch?v=K8LQSvtjcEo), which is more lenient. But for educational purposes, we'll use the Bonferroni correction, since it's easier to understand on an inuitive level.)

Now we can use our `p_values_bonf` and `log2_FCs` arrays to select a list of differentially expressed genes (DEGs) between the two conditions. The threshold choice is somewhat arbitrary here. I'm going to use `p_values_bonf <= 0.05` for statistical significance, and `np.abs(log2_FCs) >= 2` to select genes with a very large effect size (the IPF mean must be greater than 4x or less than 1/4 the control mean).

This `np.abs(log2_FCs) >= 2` cutoff is probably too strict, and in a real-life scenario it would probably make more sense to use `np.abs(log2_FCs) >= 1`. But for this dataset,` np.abs(log2_FCs) >= 1` yields quite a lot of DEGs, so for the purposes of this demo I'm trying to narrow down the list to make the next plot we're going to do easier.

(**NOTE**: sometimes the combination of p-values and log2_FC like this is called biological significance, since it requires statistical significance (determined by the p-value) as well as biological relevance (determined by the log2_FC, which measures the effect size).


In [25]:
to_keep = (p_values_bonf <= 0.05) & (np.abs(log2_FCs) >= 2)

sig_genes = genes[to_keep]
sig_log2_FCs = log2_FCs[to_keep]
sig_p_values_bonf = p_values_bonf[to_keep]
sig_data = data[to_keep, :]


Some quick checks to see how many DEGs this yielded:

In [26]:
print(f"Number of DEGs: {len(sig_genes)}")
print(f"Total number of genes: {len(genes)}")
print(f"DEG percentage: {len(sig_genes) / len(genes) * 100}")
print()
print(f"Number of upregulated DEGs: {np.sum(sig_log2_FCs>0)}")
print(f"Number of downregulated DEGs: {np.sum(sig_log2_FCs<0)}")


Number of DEGs: 79
Total number of genes: 11097
DEG percentage: 0.7119041182301523

Number of upregulated DEGs: 65
Number of downregulated DEGs: 14


if we want, we can also print out the list of DEGs to take a look. This might also be a good time to check [Enrichr](https://maayanlab.cloud/Enrichr) to see if there's anything interesting about them.

*Upregulated Genes*

In [27]:
for i in range(len(sig_genes)):
  if sig_log2_FCs[i] > 0:
    print(sig_genes[i])

PROM1
CYP24A1
CP
CDH3
COL17A1
ATP12A
FAT2
SLC4A11
DERL3
VSIG1
CD79A
COMP
COL1A1
COL7A1
FHL2
TSPAN1
MUC5B
SPP1
IL13RA2
COL10A1
BPIFB1
PDLIM4
TNS4
FCRLA
JCHAIN
POSTN
ERN2
TMPRSS4
STRA6
GPR87
FCRL5
MUC4
CXCL14
FAM83A
PLEKHS1
SYT8
THY1
CXCL13
MS4A1
SCGB3A1
CTHRC1
CLDN2
GREM1
UGT1A6
COL3A1
SPRR1A
MZB1
KRT15
LGALS7B
B3GNT3
MUC16
FDCSP
ALDH1A3
COL14A1
PLA2G2A
S100A2
BPIFA1
IGFL2
LGALS7
SERPINB4
DIO2
IGLL5
AC136428.1
MSMB
PRSS2


*Downregulated Genes*

In [28]:
for i in range(len(sig_genes)):
  if sig_log2_FCs[i] < 0:
    print(sig_genes[i])

PRX
SLC6A4
MYRF
FCN3
GGTLC1
ITLN2
DEFA4
BTNL9
MS4A15
CA4
RTKN2
AGER
DEFA1
DEFA1B


Finally, we'll save these results to CSV files -- one for the list of downregulated DEGs.

In [30]:
import os
import csv

# Create output directory if it does not exist
os.makedirs("./output", exist_ok=True)
# saving upregulated genes
outputFile = "./output/upregulated_DEGs.csv"
with open(outputFile, mode = 'w', newline='') as file:
  writer = csv.writer(file)
  writer.writerow(["gene", "p_value", "log2_FC"])
  
  for i in range(len(sig_genes)):
    if sig_log2_FCs[i] > 0:
      writer.writerow([sig_genes[i], sig_p_values_bonf[i], sig_log2_FCs[i]])
      
# saving downregulated genes
outputFile = "./output/downregulated_DEGs.csv"
with open(outputFile, mode = 'w', newline='') as file:
  writer = csv.writer(file)
  writer.writerow(["gene", "p_value", "log2_FC"])
  
  for i in range(len(sig_genes)):
    if sig_log2_FCs[i] < 0:
      writer.writerow([sig_genes[i], sig_p_values_bonf[i], sig_log2_FCs[i]])
